In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

In [3]:
import pandas as pd
import numpy as np
import re
import zipfile
import pickle

![](../images/项目流程图.png)

In [4]:
root = os.getcwd()
# input file
ZIP_DATA = os.path.join(root, 'data', '百度题库.zip')  # 要解压的文件
STOPWORDS = os.path.join(root,  'data', 'stopwords.txt')

# output file path
TRAIN_TSV = os.path.join(root,  'data', 'train.tsv')  # BERT的数据文件
DEV_TSV = os.path.join(root, 'data', 'dev.tsv')
TEST_TSV = os.path.join(root, 'data', 'test.tsv')

TOKENIZER_BINARIZER = os.path.join(root, 'data', 'tokenizer_binarizer.pickle')
X_NPY = os.path.join(root,  'data', 'x.npy')  # testcnn 和 transformer的数据文件
Y_NPY = os.path.join(root,  'data', 'y.npy')

## 解压

In [4]:
# 解压数据
def unzip_data():
    """
    :param file: 要解压的文件夹的路径
    :param unzip_dir: 解压到哪里
    :return: 解压后产生的文件夹的路径
    """
    with zipfile.ZipFile(ZIP_DATA, 'r') as z:
        z.extractall(os.path.join(root,  'data'))
        print("已将压缩包解压至{}".format(z.filename.rstrip('.zip')))
        return z.filename.rstrip('.zip')

In [5]:
data_path = unzip_data()

已将压缩包解压至E:\GitHub\Text-Classification\data\百度题库


## 整合数据

![](../images/原始数据概况.jpg)

这个数据集包含4个科目的题目，每门科目下又有不同的主题。如**历史-古代史(1000)** 括号内的数字表示有1000道题目

同时，每道题目有许多知识点

In [6]:
exm_file = os.path.join(data_path, '高中_历史', 'origin', '古代史.csv')  # 举个栗子
exm = pd.read_csv(exm_file)
exm.head(2)  # 展示数据前两行

,web-scraper-order,web-scraper-start-url,item
0,1566523436-2497,https://study.baidu.com/tiku,[题目]\n据《左传》记载，春秋后期鲁国大夫季孙氏的家臣阳虎独掌权柄后，标榜要替鲁国国君整肃...
1,1566523436-2506,https://study.baidu.com/tiku,[题目]\n秦始皇统一六国后创制了一套御玺。如任命国家官员，则封印“皇帝之玺”；若任命四夷的...


In [7]:
# 第一题的内容
exm.loc[0, 'item']

'[题目]\n据《左传》记载，春秋后期鲁国大夫季孙氏的家臣阳虎独掌权柄后，标榜要替鲁国国君整肃跋扈的大夫，此举不仅得不到知礼之士的赞成，反而受到批评。因为此举（  ）A. 挑战了宗法制度B. 损害了大夫利益C. 冲击了天子权威D. 不符合周礼规定题型: 单选题|难度: 一般|使用次数: 0|纠错复制收藏到空间加入选题篮查看答案解析答案：D解析：阳虎的身份是鲁国大夫、季孙氏的家臣，按周礼的规定，他效忠于季孙氏，而他标榜为鲁国国君整肃大夫即是僭越，所以受到批评，故违背了周礼，故选择D项。宗法制度以血缘为核心，故A项与此无关，排除；B项与题意无关，排除；材料的事件涉及鲁国国内，与周天子权威无关，排除C项。知识点：\n[知识点：]\n“重农抑商”政策,郡县制,夏商两代的政治制度,中央官制——三公九卿制,皇帝制度'

看上面第一题的内容：**结尾部分**会有知识点，所以我们需要提取出知识点。用一个list装起来
> “\n[知识点：]\n“重农抑商”政策,郡县制,夏商两代的政治制度,中央官制——三公九卿制,皇帝制度'”

`[“重农抑商”政策, 郡县制, 夏商两代的政治制度, 中央官制——三公九卿制, 皇帝制度]`

In [8]:
def combine_data(data_path):
    """
    把四门科目内的所有文件合并
    """
    r = re.compile(r'\[知识点：\]\n(.*)')  # 用来寻找知识点的正则表达式
    r1 = re.compile(r'纠错复制收藏到空间加入选题篮查看答案解析|\n|知识点：|\s|\[题目\]')  # 简单清洗
    
    data = []
    for root, dirs, files in os.walk(data_path):
        if files:  # 如果文件夹下有csv文件
            for f in files:
                subject = re.findall('高中_(.{2})', root)[0]
                topic = f.strip('.csv')
                tmp = pd.read_csv(os.path.join(root, f))  # 打开csv文件
                tmp['subject'] = subject  # 主标签：科目
                tmp['topic'] = topic  # 副标签：科目下主题
                tmp['knowledge'] = tmp['item'].apply(lambda x: r.findall(x)[0].replace(',', ' ') if r.findall(x) else '')
                tmp['item'] = tmp['item'].apply(lambda x: r1.sub('', r.sub('', x)))
                data.append(tmp)
                
    data = pd.concat(data).rename(columns={'item': 'content'}).reset_index(drop=True)
    # 删掉多余的两列
    data.drop(['web-scraper-order', 'web-scraper-start-url'], axis=1, inplace=True)
    return data

In [9]:
df = combine_data(data_path)
df.sample(3)

,content,subject,topic,knowledge
28772,细胞内糖分解代谢过程如图，下列叙述错误的是（）A植物细胞能进行过程①和③或过程①和④B真核细...,生物,稳态与环境,细胞免疫的概念和过程 无氧呼吸的概念与过程 无氧呼吸的类型 有氧呼吸的三个阶段
28635,正常情况下，人体进食后血液内胰岛素含量和胰高血糖素的含量变化情况分别是（）A.减少，增加B....,生物,稳态与环境,人体的体温调节 人体水盐平衡调节 血糖平衡的调节
8993,北京和广州两地的自转角速度和线速度相比较，正确的叙述是（）A两地的角速度和线速度都相同B两地...,地理,宇宙中的地球,地球运动的基本形式


## 提取标签
先来看一下知识点出现的频率如何

In [10]:
from collections import Counter
knowledges = ' '.join(df['knowledge']).split()  # 合并
knowledges = Counter(knowledges)
print('一共有%d个标签' % len(knowledges))

一共有919个标签


In [11]:
knowledges.most_common()[:5]  # 频率最高的5个

[('人工授精、试管婴儿等生殖技术', 4402),
 ('生物性污染', 4402),
 ('避孕的原理和方法', 4402),
 ('遗传的细胞基础', 2487),
 ('遗传的分子基础', 2455)]

In [12]:
knowledges.most_common()[-5:]  # 频率最低的5个

[('酶的发现历程', 1),
 ('植物色素的提取', 1),
 ('探究水族箱（或鱼缸）中群落的演替', 1),
 ('证明DNA是主要遗传物质的实验', 1),
 ('生物多样性形成的影响因素', 1)]

可以看到这个差别还是很大的，如果把所有低频标签也考虑了，分类效果肯定不好，因为在划分训练测试集的时候很可能样本只出现在训练集或只出现在测试集，于是下面过滤掉一些低频标签。我这里取的是1%。即**过滤掉出现次数低于样本数的1%的标签**

In [13]:
def extract_label(df, freq=0.01):
    """

    :param df: 合并后的数据集
    :param freq: 要过滤的标签占样本数量的比例
    :return: DataFrame
    """
    knowledges = ' '.join(df['knowledge']).split()  # 合并
    knowledges = Counter(knowledges)
    k = int(df.shape[0] * freq)  # 计算对应频率知识点出现的次数
    print('过滤掉出现次数少于 %d 次的标签' % k)
    top_k = {i for i in knowledges if knowledges[i] > k}  # 过滤掉知识点出现次数小于k的样本
    df.knowledge = df.knowledge.apply(lambda x: ' '.join([label for label in x.split() if label in top_k]))
    df['label'] = df[['subject', 'topic', 'knowledge']].apply(lambda x: ' '.join(x), axis=1)
    
    return df[['label', 'content']]

In [14]:
df = extract_label(df)
df.sample(2)

过滤掉出现次数少于 298 次的标签


,label,content
17241,生物 分子与细胞 生命活动离不开细胞,下列关于人体细胞的叙述，错误的是（）A.人的正常体细胞的分裂次数是有限的B.自由基攻击蛋白质...
25766,生物 稳态与环境,下列叙述中，不属于种群空间特征描述的是（）A.斑马在草原上成群活动B.每毫升河水中有9个大肠...


## 对于bert 的预处理
从这里就开始产生分支了，由于bert自带了文本预处理的工具，所以这里只需要按照bert读取文件的方式生成训练、验证、测试集即可

In [6]:
from sklearn.model_selection import train_test_split

In [16]:
def create_bert_data(df, small=False):
    """
    如果small=True：是因为自己的电脑太菜，就用比较小的数据量在本地实现模型
    该函数给bert模型划分了3个数据集
    """
    df['content'] = df['content'].apply(lambda x:x.replace(' ', ''))
    if small:
        print('use small dataset to test my local bert model really work')
        train = df.sample(128)
        dev = df.sample(64)
        test = df.sample(64)
    else:
        train, test = train_test_split(df, test_size=0.2, random_state=2020)
        train, dev = train_test_split(train, test_size=0.2, random_state=2020)

    print('preprocess for bert!')
    print('create 3 tsv file(train, dev, test) in %s' % (os.path.join(root,  'data')))
    train.to_csv(TRAIN_TSV, index=None, sep='\t')
    dev.to_csv(DEV_TSV, index=None, sep='\t')
    test.to_csv(TEST_TSV, index=None, sep='\t')

In [17]:
create_bert_data(df)

preprocess for bert!
create 3 tsv file(train, dev, test) in E:\GitHub\Text-Classification\data


## TestCNN和Transformer的预处理

接下的数据预处理流程：
![](../images/数据预处理part2.jpg)

### **去标点**，**切词**，**去停用词**
合并到`sentence_preprocess`这个函数

In [18]:
import jieba

In [19]:
def load_stopwords():
    return {line.strip() for line in open(STOPWORDS, encoding='UTF-8').readlines()}

def sentence_preprocess(sentence):
    # 去标点
    r = re.compile("[^\u4e00-\u9fa5]+|题目")
    sentence = r.sub("", sentence)  # 删除所有非汉字字符
    
    # 切词
    words = jieba.cut(sentence, cut_all=False)  
    
    # 去停用词
    stop_words = load_stopwords()
    words = [w for w in words if w not in stop_words]
    return words

def df_preprocess(df):
    df.content = df.content.apply(sentence_preprocess)
    return df

In [20]:
# 展示一下结果
sentence = '据《左传》记载，春秋后期鲁国大夫季孙氏的家臣阳虎独掌权柄后，标榜要替鲁国国君整肃跋扈的大夫'
words = sentence_preprocess(sentence)
print(words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Light\AppData\Local\Temp\jieba.cache
Loading model cost 0.986 seconds.
Prefix dict has been built succesfully.


['左传', '记载', '春秋', '后期', '鲁国', '大夫', '季孙氏', '家臣', '阳虎', '独掌', '权柄', '后', '标榜', '鲁国', '国君', '整肃', '跋扈', '大夫']


In [21]:
%%time
df = df_preprocess(df)  # 40s

Wall time: 39.3 s


### 文字转token, padding,划分数据集
这里使用keras和sklearn的工具包
- `Tokenizer` 用于把文本转换成数字
- `pad_sequences` 用于对齐文本
- `MultiLabelBinarizer` 用于把标签转化为0-1

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MultiLabelBinarizer

#### Tokenizer演示

In [23]:
tokenizer = Tokenizer(num_words=5, oov_token="<UNK>")
words = [['我们', '今天', '一起', '学习'],
         ['我们', '今天', '玩'],
         ['他们', '学习']]

In [24]:
tokenizer.fit_on_texts(words)
seqs = tokenizer.texts_to_sequences(words)
seqs

[[2, 3, 1, 4], [2, 3, 1], [1, 4]]

In [25]:
tokenizer.word_index

{'<UNK>': 1, '我们': 2, '今天': 3, '学习': 4, '一起': 5, '玩': 6, '他们': 7}

In [26]:
tokenizer.index_word

{1: '<UNK>', 2: '我们', 3: '今天', 4: '学习', 5: '一起', 6: '玩', 7: '他们'}

#### pad_sequences演示

In [27]:
# 长的截断，短的补0
pad_sequences(seqs, maxlen=3, padding='post', truncating='post')

array([[2, 3, 1],
       [2, 3, 1],
       [1, 4, 0]])

#### MultiLabelBinarizer演示

In [28]:
mlb = MultiLabelBinarizer()
labels = [['A', 'B'],
          ['A', 'D'],
          ['B', 'C', 'D'],
          ['C']]
mlb.fit_transform(labels)

array([[1, 1, 0, 0],
       [1, 0, 0, 1],
       [0, 1, 1, 1],
       [0, 0, 1, 0]])

#### 编写代码

演示完毕后开始编写代码，目的是使`df`里的文本全部变成数值型数据，为了后续给模型训练

In [29]:
df.head(2)

,label,content
0,历史 古代史 “重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度,"[左传, 记载, 春秋, 后期, 鲁国, 大夫, 季孙氏, 家臣, 阳虎, 独掌, 权柄, ..."
1,历史 古代史 “重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度,"[秦始皇, 统一, 六国后, 创制, 一套, 御玺, 任命, 国家, 官员, 封印, 皇帝,..."


In [30]:
def create_testcnn_data(df, num_words=50000, maxlen=128):
    
    # 对于label处理
    mlb = MultiLabelBinarizer()
    y = mlb.fit_transform(df.label.apply(lambda x: x.split()))
    
    # 对content处理
    tokenizer = Tokenizer(num_words=num_words, oov_token="<UNK>")
    tokenizer.fit_on_texts(df.content.tolist())
    x = tokenizer.texts_to_sequences(df.content)
    x = pad_sequences(x, maxlen=maxlen, padding='post', truncating='post')   # padding
    
    # 保存数据
    np.save(X_NPY, x)
    np.save(Y_NPY, y)
    print('已创建并保存x,y至：\n {} \n {}'.format(X_NPY, Y_NPY))
    
    # 同时还要保存tokenizer和 multi_label_binarizer
    # 否则训练结束后无法还原把数字还原成文本
    tb = {'tokenizer': tokenizer, 'binarizer': mlb}  # 用个字典来保存
    with open(TOKENIZER_BINARIZER, 'wb') as f:
        pickle.dump(tb, f)
    print('已创建并保存tokenizer和binarizer至：\n {}'.format(TOKENIZER_BINARIZER))

In [2]:
def load_testcnn_data():
    """
    如果分开保存，那要保存6个文件太麻烦了。
    所以采取读取之后划分数据集的方式
    """
    # 与之前的bert同步
    x = np.load(X_NPY)
    y = np.load(Y_NPY)
    
    # 与之前bert的划分方式统一
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=2020)
    train_x, dev_x, train_y, dev_y = train_test_split(train_x, train_y, test_size=0.2, random_state=2020)
    
    return train_x, dev_x, test_x, train_y, dev_y, test_y

In [32]:
# 读取tokenizer 和 binarizer
def load_tokenizer_binarizer():
    with open(TOKENIZER_BINARIZER, 'rb') as f:
        tb = pickle.load(f)
    return tb['tokenizer'], tb['binarizer']

In [33]:
create_testcnn_data(df)

已创建并保存x,y至：
 E:\GitHub\Text-Classification\data\x.npy 
 E:\GitHub\Text-Classification\data\y.npy
已创建并保存tokenizer和binarizer至：
 E:\GitHub\Text-Classification\data\tokenizer_binarizer.pickle


In [7]:
train_x, dev_x, test_x, train_y, dev_y, test_y = load_testcnn_data()
train_x.shape, dev_x.shape, test_x.shape, train_y.shape, dev_y.shape, test_y.shape

((19080, 128), (4770, 128), (5963, 128), (19080, 97), (4770, 97), (5963, 97))

In [9]:
train_y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
train_x

array([[ 3717,   627,  1846, ...,    59,    51,    52],
       [   47,   327,    62, ...,     0,     0,     0],
       [17430, 35229,   932, ...,   173,   218,   116],
       ...,
       [ 7211,    64,    35, ...,   300,     3,   113],
       [ 3448,  6444,   424, ...,   306,     0,     0],
       [ 3600,    58,   287, ...,     0,     0,     0]])

In [35]:
tokenizer, mlb = load_tokenizer_binarizer()

## 回顾整个数据预处理流程

![](../images/项目流程图.png)